In [ ]:
import random
import re
from math import log
from collections import Counter

# Read text from "train.txt" file
with open('train.txt', 'r') as file:
    corpus = file.read()

# Remove special characters (Except '.', ',', '?') and numbers
corpus = re.sub(r'[^a-zA-Z.,? ]', '', corpus)

# Convert to lowercase
corpus = corpus.lower()

# Tokenize the text into words
words = corpus.split()

print(words)

# Calculate unigram frequencies
word_freq = Counter(words)

#Unkown Word Handling
# Define a threshold for rare words (you can adjust this threshold)
threshold = 10

# Replace rare words with '<UNK>'
words = ['<UNK>' if word_freq[word] < threshold else word for word in words]

# Calculate bigram frequencies
bigrams = [(words[i], words[i + 1]) for i in range(len(words) - 1)]
bigram_freq = Counter(bigrams)

print(bigram_freq)

"""
#Unknown word handling - Another approach
# Define a threshold for rare bigrams (you can adjust this threshold)
bigram_threshold = 5

# Replace bigrams occurring less than the threshold with ('<UNK>', '<UNK>')
bigrams = [('<UNK>', '<UNK>') if bigram_freq[bigram] < bigram_threshold else bigram for bigram in bigrams]
bigram_freq = Counter(bigrams)

print(bigram_freq)
"""

# Compute the probability distribution of bigrams (bigram model)
default_prob = 1e-1  # Small constant probability for unknown bigrams

bigram_probs = {}

for (prev_word, word), freq in bigram_freq.items():
    if prev_word == '<UNK>':
        # Handle <UNK> as a special case
        bigram_probs[(prev_word, word)] = default_prob
    else:
        bigram_probs[(prev_word, word)] = freq / word_freq[prev_word]

print("Unsmoothed probs:")
print(bigram_probs)

# Laplace Smoothing for Bigrams
class LaplaceBigramSmoothing:
    def __init__(self, vocabulary, n_gram_counts, k=1):
        self.vocabulary = vocabulary
        self.n_gram_counts = n_gram_counts
        self.k = k

    def calculate_smoothed_probability(self, n_gram):
        # Calculate the count of the bigram and the count of the unigram prefix
        n_gram_count = self.n_gram_counts.get(n_gram, 0)
        prefix_count = self.n_gram_counts.get((n_gram[0],), 0)

        # Calculate the smoothed probability using Laplace smoothing
        # Add-k to both the numerator and denominator
        smoothed_prob = (n_gram_count + self.k) / (prefix_count + (self.k * len(self.vocabulary)))

        return smoothed_prob

# Create LaplaceBigramSmoothing instance with k = 1
laplace_bigram_smoothing = LaplaceBigramSmoothing(words, bigram_freq, k=1)

# Calculate Laplace-smoothed bigram probabilities
laplace_bigram_probs = {}
for (prev_word, word), freq in bigram_freq.items():
    smoothed_prob = laplace_bigram_smoothing.calculate_smoothed_probability((prev_word, word))
    laplace_bigram_probs[(prev_word, word)] = smoothed_prob

print("Laplace probs:")
print(laplace_bigram_probs)

# Add-K Smoothing for Bigrams
class AddKBigramSmoothing:
    def __init__(self, n_gram_counts, k=1):
        self.n_gram_counts = n_gram_counts
        self.k = k

    def calculate_smoothed_probability(self, n_gram):
        # Calculate the count of the bigram
        n_gram_count = self.n_gram_counts.get(n_gram, 0)

        # Calculate the total count of bigrams with the same prefix
        total_prefix_count = sum(count for gram, count in self.n_gram_counts.items() if gram[:-1] == n_gram[:-1])

        # Calculate the smoothed probability using Add-K smoothing
        # Add 'k' to both the numerator and denominator
        smoothed_prob = (n_gram_count + self.k) / (total_prefix_count + (self.k * len(self.n_gram_counts)))

        return smoothed_prob

# Create AddKBigramSmoothing instance with k = 1
addk_bigram_smoothing = AddKBigramSmoothing(bigram_freq, k=1)

# Calculate Add-K smoothed bigram probabilities
addk_bigram_probs = {}
for (prev_word, word), freq in bigram_freq.items():
    smoothed_prob = addk_bigram_smoothing.calculate_smoothed_probability((prev_word, word))
    addk_bigram_probs[(prev_word, word)] = smoothed_prob

print("Add-K probs:")
print(addk_bigram_probs)

"""
# Generating sentences using the bigram model
def generate_bigram_sentence():
    sentence = []
    while True:
        if not sentence:
            # Start with a random word as the first word
            word = random.choice(words)
            sentence.append(word)
        else:
            # Select the next word based on the previous word (bigram)
            prev_word = sentence[-1]
            next_word_candidates = [bigram[1] for bigram in laplace_bigram_probs if bigram[0] == prev_word]
            if not next_word_candidates:
                break  # If there are no valid next words, end the sentence
            # Use a small constant for unknown bigrams
            word = random.choices(next_word_candidates, [laplace_bigram_probs.get((prev_word, w), 1e-1) for w in next_word_candidates])[0]
            sentence.append(word)

        if word in ('.', '?'):
            break

    return ' '.join(sentence)

for _ in range(10):
    print(generate_bigram_sentence())
"""

# Read text from "validation.txt" file
with open('val.txt', 'r') as file:
    validation_corpus = file.read()

# Preprocessing: Remove special characters and numbers
validation_corpus = re.sub(r'[^a-zA-Z.,? ]', '', validation_corpus)

# Convert to lowercase and tokenize the validation text into words
validation_corpus = validation_corpus.lower()
validation_words = validation_corpus.split()

# Create a set of known words based on the training data
known_words = set(word_freq.keys())

# Handle unknown words in the validation data
validation_words = ['<UNK>' if word not in known_words or word_freq[word] < threshold else word for word in validation_words]

# Calculate perplexity for the bigram model
def calculate_unsmoothed_bigram_perplexity():
    total_log_prob = 0.0
    N = len(validation_words)

    for i in range(1, N):
        prev_word = validation_words[i - 1]
        word = validation_words[i]

        # Calculate the bigram probability
        bigram_prob = bigram_probs.get((prev_word, word), 1e-1)  # Use '<UNK>' for unknown bigrams
        total_log_prob += -1 * (1 / N) * log(bigram_prob, 2)  # Using base-2 logarithm

    perplexity = pow(2, total_log_prob)
    return perplexity

def calculate_laplace_bigram_perplexity():
    total_log_prob = 0.0
    N = len(validation_words)

    for i in range(1, N):
        prev_word = validation_words[i - 1]
        word = validation_words[i]

        # Calculate the bigram probability
        bigram_prob = laplace_bigram_probs.get((prev_word, word), 1e-1)  # Use '<UNK>' for unknown bigrams
        total_log_prob += -1 * (1 / N) * log(bigram_prob, 2)  # Using base-2 logarithm

    perplexity = pow(2, total_log_prob)
    return perplexity

def calculate_addk_bigram_perplexity():
    total_log_prob = 0.0
    N = len(validation_words)

    for i in range(1, N):
        prev_word = validation_words[i - 1]
        word = validation_words[i]

        # Calculate the bigram probability
        bigram_prob = addk_bigram_probs.get((prev_word, word), 1e-1)  # Use '<UNK>' for unknown bigrams
        total_log_prob += -1 * (1 / N) * log(bigram_prob, 2)  # Using base-2 logarithm

    perplexity = pow(2, total_log_prob)
    return perplexity

# Calculate and print perplexity
unsmoothed_bigram_perplexity = calculate_unsmoothed_bigram_perplexity()
laplace_bigram_perplexity = calculate_laplace_bigram_perplexity()
addk_bigram_perplexity = calculate_addk_bigram_perplexity()

print(f"Unsmoothed Bigram Perplexity: {unsmoothed_bigram_perplexity}")
print(f"Laplace Bigram Perplexity: {laplace_bigram_perplexity}")
print(f"Add-K Bigram Perplexity: {addk_bigram_perplexity}")

['i', 'booked', 'two', 'rooms', 'four', 'months', 'in', 'advance', 'at', 'the', 'talbott', '.', 'we', 'were', 'placed', 'on', 'the', 'top', 'floor', 'next', 'to', 'the', 'elevators', ',', 'which', 'are', 'used', 'all', 'night', 'long', '.', 'when', 'speaking', 'to', 'the', 'front', 'desk', ',', 'i', 'was', 'told', 'that', 'they', 'were', 'simply', 'honoring', 'my', 'request', 'for', 'an', 'upper', 'floor', ',', 'which', 'i', 'had', 'requested', 'for', 'a', 'better', 'view', '.', 'i', 'am', 'looking', 'at', 'a', 'brick', 'wall', ',', 'and', 'getting', 'no', 'sleep', '.', 'he', 'also', 'told', 'me', 'that', 'they', 'had', 'received', 'complaints', 'before', 'from', 'guests', 'on', 'the', 'th', 'floor', ',', 'and', 'were', 'aware', 'of', 'the', 'noise', 'problem', '.', 'why', 'then', 'did', 'they', 'place', 'us', 'on', 'this', 'floor', 'when', 'the', 'hotel', 'is', 'not', 'totally', 'booked', '?', 'a', 'request', 'for', 'an', 'upper', 'floor', 'does', 'not', 'constitute', 'placing', 'some